# 🎥 Detector de Emociones Faciales - Detección en Tiempo Real

## Objetivo
Crear una aplicación que use la **webcam** para detectar rostros y clasificar emociones en tiempo real.

## Funcionamiento
1. **Capturar video** de la webcam
2. **Detectar rostros** usando Haar Cascade de OpenCV
3. **Preprocesar** el rostro (48x48, escala de grises, normalizar)
4. **Predecir emoción** con nuestro modelo CNN
5. **Mostrar resultado** en pantalla con la emoción y confianza

## Controles
- **Presiona 'q'** para salir
- **Presiona 's'** para capturar screenshot
- **ESC** también cierra la aplicación

¡Prepárate para hacer diferentes expresiones faciales! 😊😢😠😮

In [1]:
# Importar librerías
import cv2
import numpy as np
from tensorflow import keras
from pathlib import Path
import time

# Configuración
print("✅ Librerías importadas")
print(f"OpenCV version: {cv2.__version__}")

✅ Librerías importadas
OpenCV version: 4.8.1


## 🤖 Cargar Modelo Entrenado

Cargaremos el mejor modelo (V2) que entrenamos con 60% accuracy.

In [2]:
# Cargar el modelo entrenado
model_path = Path('../models/best_model_v2.keras')

print("📂 Cargando modelo entrenado...")
if model_path.exists():
    model = keras.models.load_model(model_path)
    print("✅ Modelo cargado exitosamente")
else:
    print("❌ Error: Modelo no encontrado")
    print(f"   Busca en: {model_path.absolute()}")

# Definir emociones (mismo orden que en el entrenamiento)
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Emojis para cada emoción
emotion_emojis = {
    'angry': '😠',
    'disgust': '🤢',
    'fear': '😨',
    'happy': '😊',
    'neutral': '😐',
    'sad': '😢',
    'surprise': '😮'
}

print(f"\n🎭 Emociones a detectar: {emotions}")
print(f"📊 Modelo entrenado con 60% accuracy")

📂 Cargando modelo entrenado...
✅ Modelo cargado exitosamente

🎭 Emociones a detectar: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
📊 Modelo entrenado con 60% accuracy


## 👤 Detector de Rostros

Usaremos **Haar Cascade** de OpenCV para detectar rostros en el video.
Este es un detector clásico, rápido y eficiente para detección facial.

In [3]:
# Cargar clasificador Haar Cascade para detección de rostros
face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(face_cascade_path)

if face_cascade.empty():
    print("❌ Error: No se pudo cargar el detector de rostros")
else:
    print("✅ Detector de rostros cargado correctamente")
    print(f"📁 Ruta: {face_cascade_path}")

✅ Detector de rostros cargado correctamente
📁 Ruta: c:\Users\patri\miniconda3\envs\detector-emociones\lib\site-packages\cv2\data\haarcascade_frontalface_default.xml


## 🔧 Funciones de Procesamiento

Crearemos funciones para:
1. Preprocesar el rostro detectado (resize, normalizar)
2. Predecir la emoción
3. Dibujar resultados en la imagen

In [4]:
def preprocess_face(face_img):
    """
    Preprocesa la imagen del rostro para el modelo
    """
    # Convertir a escala de grises si es necesario
    if len(face_img.shape) == 3:
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    
    # Redimensionar a 48x48
    face_img = cv2.resize(face_img, (48, 48))
    
    # Normalizar (0-1)
    face_img = face_img / 255.0
    
    # Reshape para el modelo (batch, height, width, channels)
    face_img = face_img.reshape(1, 48, 48, 1)
    
    return face_img


def predict_emotion(face_img, model):
    """
    Predice la emoción del rostro
    """
    # Preprocesar
    processed_face = preprocess_face(face_img)
    
    # Predecir
    predictions = model.predict(processed_face, verbose=0)
    
    # Obtener emoción con mayor probabilidad
    emotion_idx = np.argmax(predictions[0])
    emotion = emotions[emotion_idx]
    confidence = predictions[0][emotion_idx]
    
    return emotion, confidence, predictions[0]


def draw_emotion_results(frame, x, y, w, h, emotion, confidence):
    """
    Dibuja el rectángulo y la emoción en el frame
    """
    # Colores según la emoción
    colors = {
        'angry': (0, 0, 255),      # Rojo
        'disgust': (0, 128, 128),  # Verde oscuro
        'fear': (128, 0, 128),     # Púrpura
        'happy': (0, 255, 0),      # Verde
        'neutral': (128, 128, 128),# Gris
        'sad': (255, 0, 0),        # Azul
        'surprise': (0, 255, 255)  # Amarillo
    }
    
    color = colors.get(emotion, (255, 255, 255))
    
    # Dibujar rectángulo alrededor del rostro
    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
    
    # Texto de la emoción con emoji
    emoji = emotion_emojis.get(emotion, '😐')
    text = f"{emoji} {emotion.upper()} ({confidence*100:.1f}%)"
    
    # Fondo para el texto
    (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
    cv2.rectangle(frame, (x, y - text_h - 10), (x + text_w, y), color, -1)
    
    # Texto
    cv2.putText(frame, text, (x, y - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return frame


print("✅ Funciones de procesamiento creadas")

✅ Funciones de procesamiento creadas


## 🎥 Detección en Tiempo Real

Esta función activará tu webcam y detectará emociones en tiempo real.

**Controles:**
- Presiona **'q'** para salir
- Presiona **'s'** para guardar screenshot
- Presiona **ESC** también cierra la aplicación

¡Prepárate para hacer diferentes caras! 😊😢😠😮

In [5]:
def run_emotion_detection():
    """
    Función principal para detección de emociones en tiempo real
    """
    # Iniciar captura de video
    print("🎥 Iniciando webcam...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Error: No se pudo acceder a la webcam")
        return
    
    print("✅ Webcam iniciada")
    print("\n📋 CONTROLES:")
    print("  - Presiona 'q' para salir")
    print("  - Presiona 's' para guardar screenshot")
    print("  - Presiona ESC para salir")
    print("\n🎭 ¡Haz diferentes expresiones faciales!")
    
    screenshot_count = 0
    
    while True:
        # Capturar frame
        ret, frame = cap.read()
        
        if not ret:
            print("❌ Error al capturar frame")
            break
        
        # Convertir a escala de grises para detección
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detectar rostros
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(48, 48)
        )
        
        # Procesar cada rostro detectado
        for (x, y, w, h) in faces:
            # Extraer región del rostro
            face_roi = gray[y:y+h, x:x+w]
            
            # Predecir emoción
            emotion, confidence, all_predictions = predict_emotion(face_roi, model)
            
            # Dibujar resultados
            frame = draw_emotion_results(frame, x, y, w, h, emotion, confidence)
        
        # Mostrar información en pantalla
        cv2.putText(frame, f"Rostros detectados: {len(faces)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.putText(frame, "Presiona 'q' para salir | 's' para screenshot", (10, frame.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Mostrar frame
        cv2.imshow('Detector de Emociones Faciales', frame)
        
        # Controles de teclado
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # 'q' o ESC
            print("\n👋 Cerrando aplicación...")
            break
        elif key == ord('s'):  # Guardar screenshot
            screenshot_count += 1
            filename = f'../data/screenshot_{screenshot_count}.jpg'
            cv2.imwrite(filename, frame)
            print(f"📸 Screenshot guardado: {filename}")
    
    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Aplicación cerrada correctamente")


print("✅ Función de detección en tiempo real creada")
print("\n🚀 Para iniciar la detección, ejecuta la siguiente celda")

✅ Función de detección en tiempo real creada

🚀 Para iniciar la detección, ejecuta la siguiente celda


## 🚀 ¡Iniciar Detección!

Ejecuta la siguiente celda para activar tu webcam y empezar a detectar emociones.

**Tips:**
- Asegúrate de tener buena iluminación
- Mira directamente a la cámara
- Prueba diferentes expresiones: sonríe, enójate, sorpréndete
- El modelo funciona mejor con expresiones claras y exageradas

In [ ]:
# ¡INICIAR DETECCIÓN DE EMOCIONES!
print("=" * 60)
print("🎥 DETECTOR DE EMOCIONES FACIALES EN TIEMPO REAL")
print("=" * 60)

run_emotion_detection()

🎥 DETECTOR DE EMOCIONES FACIALES EN TIEMPO REAL
🎥 Iniciando webcam...
✅ Webcam iniciada

📋 CONTROLES:
  - Presiona 'q' para salir
  - Presiona 's' para guardar screenshot
  - Presiona ESC para salir

🎭 ¡Haz diferentes expresiones faciales!
